In [2]:
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline
from diffusers.utils import load_image
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model
from skimage import io, img_as_float

import os
import cv2
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import tensorflow as tf

import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
import tensorflow as tf
from lpips import LPIPS
import warnings
from numpy import linalg
from skimage import io, color

/home/as05412/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2024-04-21 13:40:52.763822: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 13:40:52.791892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: 

# Generating Prediction Images

In [ ]:
def predict(pipeline,image_path):
    
    image = load_image(image_path)
    
    # Load metadata CSV
    metadata = pd.read_csv('data_preparation/metadata.csv')

    # Extract the first 11 letters of the folder of the image
    folder_name = image_path.split("/")[-2][:11]
    print(folder_name)

    # Check if the folder name is present in the 'Name' column of metadata
    match = metadata[metadata['Name'].str.startswith(folder_name)]

    if not match.empty:
        # Extract information from the matched row
        latitude = match['Latitude'].values[0]
        longitude = match['Longitude'].values[0]
        location_name = match['Location_name'].values[0]

        # Update the pipeline with the extracted values
        print(f"Latitude: {latitude}, Longitude: {longitude}, Location name: {location_name} predict green carbonates if present")
        image = pipeline(f"Longitude: {longitude}, Latitude: {latitude}, Location name: {location_name} predict green carbonates", image=image).images[0]
    else:
        # If no match is found, use the default pipeline
        image = pipeline("predict green carbonates", image=image).images[0]
        print('predict green carboates if present')

    return image

In [ ]:
def predict_without_mtda(pipeline,image_path):
    
    image = load_image(image_path)  
    
    image = pipeline("predict green carbonates", image=image).images[0]
    return image

In [ ]:
def predict_for_directory(pipeline,folder_path):
    # Walk through the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # If the file is 'original_image.png'
            if file == 'original_image.png':
                # Open the image
                
                input_image_path = os.path.join(root, file)
                print('input image path ', input_image_path)
                # input_image = load_image(input_image_path)

                # Apply the predict function
                output_image = predict_without_mtda(pipeline,input_image_path)

                # Save the output image as 'predicted_image.png' in the same subfolder
                #print(root)
                output_image_path = os.path.join(root, 'ip_pnr_without_mtda_pnr_mtda_model_predicted_image.png')
                output_image.save(output_image_path)

In [ ]:
# Load appropriate model
model_id = "crism_ip_pnr_with_mtda"
pipeline = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16).to("cuda")

In [ ]:
predict_for_directory(pipeline,'dataset_pnr/crism_dataset/test')

# Metric Calculation functions

In [3]:
def calculate_rmse(original, predicted):
    return np.sqrt(mean_squared_error(original.flatten(), predicted.flatten()))

def calculate_pcc(original, predicted):
    original_flat = original.flatten()
    predicted_flat = predicted.flatten()
    correlation, _ = pearsonr(original_flat, predicted_flat)
    return correlation

def calculate_cosine_similarity(original, predicted):
    original_flat = original.flatten()
    predicted_flat = predicted.flatten()
    similarity = cosine_similarity([original_flat], [predicted_flat])
    return similarity[0][0]


def calculate_ssim(image1_path, image2_path):
    # Load the images
    img1 = img_as_float(io.imread(image1_path))
    img2 = img_as_float(io.imread(image2_path))
    
    img1 = cv2.resize(img1, (512, 512))
    img2 = cv2.resize(img2, (512, 512))
    # Validate if both images have same dimensions
    if img1.shape != img2.shape:
        raise ValueError("Both input images should have the same dimensions")

    # Calculate SSIM for each channel separately and average them
    ssim_index = (ssim(img1[..., 0], img2[..., 0], win_size=5, data_range=img1[..., 0].max() - img1[..., 0].min()) +
                  ssim(img1[..., 1], img2[..., 1], win_size=5, data_range=img1[..., 1].max() - img1[..., 1].min()) +
                  ssim(img1[..., 2], img2[..., 2], win_size=5, data_range=img1[..., 2].max() - img1[..., 2].min())) / 3

    return ssim_index

common_size = (512, 512)



def calculate_lpips(edited_image, predicted_image):
    # Suppress warnings
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)

        # Convert images to PyTorch tensors
        edited_image_tensor = torch.from_numpy(edited_image.transpose(2, 0, 1)).float()
        predicted_image_tensor = torch.from_numpy(predicted_image.transpose(2, 0, 1)).float()
        
        # Initialize LPIPS model
        lpips_model = LPIPS(net='alex', version='0.1', spatial=False)
        
        # Calculate LPIPS value
        lpips_value = lpips_model(edited_image_tensor, predicted_image_tensor)
    
    return lpips_value.item()
   

def calculate_lpips_folder(root_folder, edited_image_name):

    fid_values = []
    lpips_values = []
    mcd_values = []

    for subfolder in os.listdir(root_folder):
        subfolder_path = os.path.join(root_folder, subfolder)

        original_image_path = os.path.join(subfolder_path, 'original_image.png')
        edited_image_path = os.path.join(subfolder_path, 'edited_image.png')
        predicted_image_path = os.path.join(subfolder_path, edited_image_name)

        original_image = cv2.imread(original_image_path, cv2.IMREAD_COLOR)
        edited_image = cv2.imread(edited_image_path, cv2.IMREAD_COLOR)
        predicted_image = cv2.imread(predicted_image_path, cv2.IMREAD_COLOR)

        # Resize images to a common size
        original_image = cv2.resize(original_image, common_size)
        edited_image = cv2.resize(edited_image, common_size)
        predicted_image = cv2.resize(predicted_image, common_size)

        # Convert NumPy arrays to PyTorch tensors
        edited_image_tensor = torch.from_numpy(edited_image.transpose(2, 0, 1)).float()
        predicted_image_tensor = torch.from_numpy(predicted_image.transpose(2, 0, 1)).float()

        # Calculate LPIPS
        lpips_model = LPIPS(net='alex',version='0.1', spatial=False)
        lpips_value = lpips_model(edited_image_tensor, predicted_image_tensor)
        lpips_values.append(lpips_value.item())

    return lpips_values


def calculate_average(values):
    return sum(values) / len(values)


In [4]:
def calculate_metrics_for_folder(root_folder,edited_image_name):
    rmse_values = []
    pcc_values = []
    cosine_similarity_values = []
    ssim_values = []
    mae_values = []
    lpips_values = []

    for subfolder in os.listdir(root_folder):
        subfolder_path = os.path.join(root_folder, subfolder)

        original_image_path = os.path.join(subfolder_path, 'original_image.png')
        edited_image_path = os.path.join(subfolder_path, 'edited_image.png')
        predicted_image_path = os.path.join(subfolder_path, edited_image_name)
        # print('path og ',original_image_path)

        original_image = cv2.imread(original_image_path, cv2.IMREAD_COLOR)
        edited_image = cv2.imread(edited_image_path, cv2.IMREAD_COLOR)
        predicted_image = cv2.imread(predicted_image_path, cv2.IMREAD_COLOR)
        
        # Resize images to a common size
        original_image = cv2.resize(original_image, common_size)
        edited_image = cv2.resize(edited_image, common_size)
        predicted_image = cv2.resize(predicted_image, common_size)

        # Calculate RMSE
        rmse_value = calculate_rmse(edited_image, predicted_image)
        rmse_values.append(rmse_value)

        # Calculate PCC
        pcc_value = calculate_pcc(edited_image, predicted_image)
        pcc_values.append(pcc_value)

        # Calculate Cosine Similarity
        cosine_similarity_value = calculate_cosine_similarity(edited_image, predicted_image)
        cosine_similarity_values.append(cosine_similarity_value)

        # Calculate SSIM
        ssim_index = calculate_ssim(edited_image_path, predicted_image_path)
        ssim_values.append(ssim_index)
        
        # Calculate LPIPS
        lpips = calculate_lpips(edited_image, predicted_image)
        lpips_values.append(lpips)


    return rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values



# Results of different dataset test sets

## Unet PNR Score

In [12]:
root_directory = 'dataset_pnr/crism_dataset/test/'
edited_image_name = 'unet_new_pnr_predicted_image.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
# average_psnr = calculate_average(psnr_values)
average_rmse = calculate_average(rmse_values)
average_pcc = calculate_average(pcc_values)
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)

# print(f"Average PSNR: {average_psnr}")
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

# Merge Results
merged_results = { 'RMSE': rmse_values, 'PCC': pcc_values, 'Cosine Similarity': cosine_similarity_values, 'SSIM': ssim_values, 'LPIPS': lpips_values}


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


...

Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Average RMSE: 6.966844170883489
Average PCC: 0.9610245125671629
Average Cosine Similarity: 0.9857119335838629
Average SSIM: 0.9148986684014626
Average LPIPS: 0.1281974742073061


In [13]:
# print(f"Average PSNR: {average_psnr}")
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 6.966844170883489
Average PCC: 0.9610245125671629
Average Cosine Similarity: 0.9857119335838629
Average SSIM: 0.9148986684014626
Average LPIPS: 0.1281974742073061


## Unet on images with high carbonate presence

In [9]:
# Example usage:
root_directory = 'CrismGreenImages'
edited_image_name = 'unet_new_pnr_predicted_image.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
average_rmse = calculate_average(rmse_values)
average_pcc = calculate_average(pcc_values)
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)




Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


...

Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [10]:
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 7.466069255763828
Average PCC: 0.9324591835555
Average Cosine Similarity: 0.9755234136797832
Average SSIM: 0.8829909278049594
Average LPIPS: 0.1695300830262048


## Unet on PCA images

In [12]:
root_directory = 'dataset_pca/crism_dataset/test'
edited_image_name = 'unet_new_pca_predicted_image.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
average_rmse = calculate_average(rmse_values)
average_pcc = calculate_average(pcc_values)
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


...

Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/as05412/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [13]:
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 10.420519512503864
Average PCC: 0.13464647858197915
Average Cosine Similarity: 0.7552037304889528
Average SSIM: 0.2619790212862138
Average LPIPS: 0.42222192050779567


## Stable diffuison on PNR dataset

### Without metadata

In [5]:

root_directory = 'dataset_pnr/crism_dataset/test'
edited_image_name = 'ip_pnr_without_mtda_pnr_mtda_model_predicted_image.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
average_rmse = calculate_average(rmse_values)
average_pcc = calculate_average(pcc_values)
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/

/home/as05412/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], 

/home/as05412/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], 

In [7]:
import math

filtered_values = [value for value in pcc_values if not math.isnan(value)]
average_pcc = calculate_average(filtered_values)
average_pcc

0.9699386078935451

In [8]:
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 7.097861521955683
Average PCC: 0.9699386078935451
Average Cosine Similarity: 0.980903162912709
Average SSIM: 0.8263126437959618
Average LPIPS: 0.07195745977562862


### With Metadata

In [6]:
root_directory = 'dataset_pnr/crism_dataset/test'
edited_image_name = 'ip_pnr_mtda_predicted_image.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
average_rmse = calculate_average(rmse_values)
#average_pcc = calculate_average(pcc_values[~np.isnan(pcc_values)])
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/

/home/as05412/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], 

/home/as05412/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], 

In [9]:
import math

filtered_values = [value for value in pcc_values if not math.isnan(value)]
average_pcc = calculate_average(filtered_values)
average_pcc


0.9744420883303383

In [10]:
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 6.743323985592821
Average PCC: 0.9744420883303383
Average Cosine Similarity: 0.9825481136805244
Average SSIM: 0.8328498730496638
Average LPIPS: 0.0671808840212286


## Stable diffusion on PCA dataset

In [7]:
root_directory = 'dataset_pca/crism_dataset/test'
edited_image_name = 'ip_pca_mtda_predicted_image.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
average_rmse = calculate_average(rmse_values)
average_pcc = calculate_average(pcc_values)
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/

/home/as05412/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], 

/home/as05412/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], 

In [13]:
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 8.730883257242247
Average PCC: 0.8803651727315699
Average Cosine Similarity: 0.9519500269534991
Average SSIM: 0.5762348226404727
Average LPIPS: 0.16891109497294204


## Stable diffusion on New Channel dataset

In [14]:
root_directory = 'dataset_newChannel/crism_dataset_newChannel/test'
edited_image_name = 'ip_predicted_newChannel.png'
rmse_values, pcc_values, cosine_similarity_values, ssim_values, lpips_values = calculate_metrics_for_folder(root_directory,edited_image_name)

# Calculate averages
average_rmse = calculate_average(rmse_values)
average_pcc = calculate_average(pcc_values)
average_cosine_similarity = calculate_average(cosine_similarity_values)
average_ssim = calculate_average(ssim_values)
average_lpips = calculate_average(lpips_values)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/as05412/.local/lib/python3.8/site-packages/lpips/weights/

In [15]:
print(f"Average RMSE: {average_rmse}")
print(f"Average PCC: {average_pcc}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")
print(f"Average SSIM: {average_ssim}")
print(f"Average LPIPS: {average_lpips}")

Average RMSE: 6.381248960081898
Average PCC: 0.9844565994695114
Average Cosine Similarity: 0.994248079766849
Average SSIM: 0.8516697849075393
Average LPIPS: 0.03940862905742092
